# 二値分類の実践例

このノートブックでは、実際のデータセットを使用してロジスティック回帰による二値分類を実践します。

## 学習目標
- 実データでのロジスティック回帰の適用
- データの前処理と特徴量エンジニアリング
- モデルの評価と解釈
- ビジネス応用の理解


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, roc_auc_score, roc_curve, confusion_matrix,
                           classification_report)
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")


## 1. データセットの読み込み

乳がんデータセットを使用して、悪性（malignant）と良性（benign）の分類を行います。


In [ ]:
# 乳がんデータセットの読み込み
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# データフレームの作成
df = pd.DataFrame(X, columns=cancer.feature_names)
df['target'] = y

print("=== データセットの基本情報 ===")
print(f"データの形状: {X.shape}")
print(f"特徴量の数: {X.shape[1]}")
print(f"サンプル数: {X.shape[0]}")
print(f"クラスの分布: {np.bincount(y)}")
print(f"クラス名: {cancer.target_names}")

# データの可視化（最初の2つの特徴量）
plt.figure(figsize=(10, 6))
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='red', label='Malignant', alpha=0.7)
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Benign', alpha=0.7)
plt.xlabel(cancer.feature_names[0])
plt.ylabel(cancer.feature_names[1])
plt.title('Breast Cancer Dataset - First Two Features')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# データの基本統計
print("\n=== データの基本統計 ===")
print(df.describe())


## 2. データの前処理


In [ ]:
# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 特徴量の標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("=== データ分割の結果 ===")
print(f"訓練データの形状: {X_train_scaled.shape}")
print(f"テストデータの形状: {X_test_scaled.shape}")
print(f"訓練データのクラス分布: {np.bincount(y_train)}")
print(f"テストデータのクラス分布: {np.bincount(y_test)}")

# 標準化前後の比較
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(X_train[:, 0], bins=30, alpha=0.7, label='Original')
plt.title('Original Data - First Feature')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 3, 2)
plt.hist(X_train_scaled[:, 0], bins=30, alpha=0.7, label='Scaled', color='orange')
plt.title('Scaled Data - First Feature')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 3, 3)
plt.boxplot([X_train[:, 0], X_train_scaled[:, 0]], labels=['Original', 'Scaled'])
plt.title('Box Plot Comparison')
plt.ylabel('Value')

plt.tight_layout()
plt.show()


## 3. モデルの訓練と評価


In [ ]:
# ロジスティック回帰モデルの訓練
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# 予測
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# 基本的な評価指標
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print("=== モデルの評価結果 ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC: {auc:.4f}")

# 混同行列の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Malignant', 'Benign'], 
            yticklabels=['Malignant', 'Benign'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# ROC曲線の描画
plt.subplot(1, 2, 2)
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 分類レポート
print("\n=== 詳細な分類レポート ===")
print(classification_report(y_test, y_pred, target_names=['Malignant', 'Benign']))


## 4. 特徴量の重要度分析


In [ ]:
# 特徴量の重要度（係数の絶対値）
feature_importance = np.abs(model.coef_[0])
feature_names = cancer.feature_names

# 重要度の順位付け
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance,
    'coefficient': model.coef_[0]
}).sort_values('importance', ascending=False)

print("=== 特徴量の重要度（上位10位） ===")
print(importance_df.head(10))

# 重要度の可視化
plt.figure(figsize=(12, 8))
top_features = importance_df.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance (|Coefficient|)')
plt.title('Top 15 Most Important Features')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 係数の分布
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(model.coef_[0], bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('Coefficient Value')
plt.ylabel('Frequency')
plt.title('Distribution of Coefficients')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(range(len(model.coef_[0])), model.coef_[0], alpha=0.7)
plt.xlabel('Feature Index')
plt.ylabel('Coefficient Value')
plt.title('Coefficients by Feature Index')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. ハイパーパラメータの最適化


In [ ]:
# グリッドサーチによるハイパーパラメータの最適化
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

grid_search = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=1000),
    param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train)

print("=== グリッドサーチの結果 ===")
print(f"最良のパラメータ: {grid_search.best_params_}")
print(f"最良のスコア: {grid_search.best_score_:.4f}")

# 最良のモデルで予測
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)
y_pred_proba_best = best_model.predict_proba(X_test_scaled)[:, 1]

# 最適化後の評価
accuracy_best = accuracy_score(y_test, y_pred_best)
auc_best = roc_auc_score(y_test, y_pred_proba_best)

print(f"\n=== 最適化後の評価 ===")
print(f"Accuracy: {accuracy_best:.4f}")
print(f"AUC: {auc_best:.4f}")

# 正則化パラメータCの影響を可視化
C_values = [0.001, 0.01, 0.1, 1, 10, 100]
train_scores = []
val_scores = []

for C in C_values:
    model = LogisticRegression(C=C, random_state=42, max_iter=1000)
    model.fit(X_train_scaled, y_train)
    
    train_score = roc_auc_score(y_train, model.predict_proba(X_train_scaled)[:, 1])
    val_score = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='roc_auc').mean()
    
    train_scores.append(train_score)
    val_scores.append(val_score)

plt.figure(figsize=(10, 6))
plt.plot(C_values, train_scores, 'o-', label='Training Score')
plt.plot(C_values, val_scores, 'o-', label='Validation Score')
plt.xscale('log')
plt.xlabel('Regularization Parameter (C)')
plt.ylabel('ROC AUC Score')
plt.title('Effect of Regularization Parameter C')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## 6. ビジネス応用の考察

### 6.1 医療診断における解釈


In [ ]:
# 予測確率の分布と解釈
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(y_pred_proba_best[y_test == 0], bins=20, alpha=0.7, label='Malignant', color='red')
plt.hist(y_pred_proba_best[y_test == 1], bins=20, alpha=0.7, label='Benign', color='blue')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Probabilities')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
# 信頼度の高い予測（確率が0.8以上または0.2以下）
high_confidence = (y_pred_proba_best >= 0.8) | (y_pred_proba_best <= 0.2)
plt.scatter(y_pred_proba_best[high_confidence], y_test[high_confidence], 
           c='green', alpha=0.7, label='High Confidence')
plt.scatter(y_pred_proba_best[~high_confidence], y_test[~high_confidence], 
           c='orange', alpha=0.7, label='Low Confidence')
plt.xlabel('Predicted Probability')
plt.ylabel('True Label')
plt.title('Prediction Confidence')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
# 誤分類の分析
misclassified = (y_pred_best != y_test)
plt.scatter(y_pred_proba_best[~misclassified], y_test[~misclassified], 
           c='green', alpha=0.7, label='Correct')
plt.scatter(y_pred_proba_best[misclassified], y_test[misclassified], 
           c='red', alpha=0.7, label='Misclassified')
plt.xlabel('Predicted Probability')
plt.ylabel('True Label')
plt.title('Classification Results')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 信頼度の統計
print("=== 予測の信頼度分析 ===")
print(f"高信頼度予測の割合: {np.mean(high_confidence):.2%}")
print(f"高信頼度予測の精度: {accuracy_score(y_test[high_confidence], y_pred_best[high_confidence]):.4f}")
print(f"低信頼度予測の精度: {accuracy_score(y_test[~high_confidence], y_pred_best[~high_confidence]):.4f}")

# 誤分類の詳細分析
print(f"\n=== 誤分類の分析 ===")
print(f"誤分類数: {np.sum(misclassified)}")
print(f"誤分類率: {np.mean(misclassified):.2%}")
print(f"誤分類の平均確率: {np.mean(y_pred_proba_best[misclassified]):.4f}")
print(f"正分類の平均確率: {np.mean(y_pred_proba_best[~misclassified]):.4f}")


## 7. 演習問題

### 演習1: 特徴量選択
上位10個の特徴量のみを使用してモデルを再訓練し、性能を比較してみましょう。

### 演習2: 決定閾値の調整
医療診断では、False Negative（見逃し）を避けることが重要です。Recallを最大化する閾値を探してみましょう。

### 演習3: クラス不均衡の対処
データセットのクラス分布を変更して、不均衡データに対する対策を実装してみましょう。


## まとめ

このノートブックでは、乳がんデータセットを使用してロジスティック回帰による二値分類を実践しました。

**学習した内容**：
- 実データでのロジスティック回帰の適用
- データの前処理と特徴量の標準化
- モデルの評価と解釈
- ハイパーパラメータの最適化
- 特徴量の重要度分析
- ビジネス応用における解釈

**重要なポイント**：
- 医療診断では確率的な出力が重要
- 特徴量の標準化が性能に大きく影響
- 正則化パラメータの調整が重要
- 解釈可能性が実務では重要

**次のステップ**：
- 多クラス分類への拡張
- より高度な特徴量エンジニアリング
- アンサンブル手法の学習
